In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
# 한글 출력을 위한 설정
from matplotlib import font_manager, rc
import platform
if platform.system()=='Darwin':
    rc('font', family='AppleGothic')
elif platform.system()=='Windows':
    font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
    rc('font', family=font_name)
# 음수 사용(마이너스 기호 깨짐 방지)
plt.rcParams['axes.unicode_minus'] = False

In [2]:
# auto-mpg 데이터 읽어오기
auto_mpg=pd.read_csv('./data/auto-mpg.csv', header=None)
# 컬럼 이름 설정하기
auto_mpg.columns=['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
                 'acceleration', 'model year', 'origin', 'name']
auto_mpg['horsepower'].replace('?',np.nan,inplace=True)
auto_mpg.dropna(subset=['horsepower'],axis=0, inplace=True)
auto_mpg['horsepower']=auto_mpg['horsepower'].astype('float')
auto_mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


## One Hot Encoding

In [5]:
# One Hot Encoding
# Horsepower 특성을 범주형으로 추가 - 3개의 영역으로 구분
# 3개의 구간으로 구분해서 개수와 경계값을 리턴받아서 저장
count,bin_dividers=np.histogram(auto_mpg['horsepower'],bins=3)
# 범주형 형태로 생성
auto_mpg['hp_bin']=pd.cut(x=auto_mpg['horsepower'],
                         bins=bin_dividers,
                         labels=['저출력', '보통출력', '고출력'],
                         include_lowest=True)
auto_mpg[['horsepower','hp_bin']].head(10)

,horsepower,hp_bin
0,130.0,보통출력
1,165.0,보통출력
2,150.0,보통출력
3,150.0,보통출력
4,140.0,보통출력
5,198.0,고출력
6,220.0,고출력
7,215.0,고출력
8,225.0,고출력
9,190.0,고출력


In [9]:
# 원 핫 인코딩 수행 - 값이 3종류이므로, 
# 3개의 특성이 만들어지고 값은 하나만 1이다.
horsepower_dummies=pd.get_dummies(auto_mpg['hp_bin'])
horsepower_dummies.head(7)

,저출력,보통출력,고출력
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0
5,0,0,1
6,0,0,1


In [12]:
from sklearn.preprocessing import LabelBinarizer
one_hot=LabelBinarizer()
one_hot.fit_transform(auto_mpg['hp_bin'])
#이름확인
one_hot.classes_

array(['고출력', '보통출력', '저출력'], dtype='<U4')

In [18]:
# 2개 이상의 특성을 가지고 원 핫 인코딩이 가능합니다.
# 2ㅐ 이상의 1이 등장할 수 있습니다.
from sklearn.preprocessing import MultiLabelBinarizer
multi_feature=[('java','c++'),('c++', 'python'), ('java', 'c#'),
            ('java', 'kotlin'), ('python', 'go'), ('python', 'r')]
one_hot_multiclass=MultiLabelBinarizer()
print(one_hot_multiclass.fit_transform(multi_feature))
print(one_hot_multiclass.classes_)

[[0 1 0 1 0 0 0]
 [0 1 0 0 0 1 0]
 [1 0 0 1 0 0 0]
 [0 0 0 1 1 0 0]
 [0 0 1 0 0 1 0]
 [0 0 0 0 0 1 1]]
['c#' 'c++' 'go' 'java' 'kotlin' 'python' 'r']


In [19]:
# 순서가 의미를 갖는 경우 - replace 함수 이용
df=pd.DataFrame({"Score":['저조','보통','보통','저조','우수','매우우수']})
scale_mapper={'저조':1,'보통':2,'우수':3,'매우우수':4}
df['encoder']=df['Score'].replace(scale_mapper)
print(df)

  Score  encoder
0    저조        1
1    보통        2
2    보통        2
3    저조        1
4    우수        3
5  매우우수        4


In [21]:
from sklearn.preprocessing import OrdinalEncoder
features=np.array([['Low',10],['Normal',20],['High',15]])
ordinal_encoder=OrdinalEncoder()
print(ordinal_encoder.fit_transform(features))
print(ordinal_encoder.categories_)

[[1. 0.]
 [2. 2.]
 [0. 1.]]
[array(['High', 'Low', 'Normal'], dtype='<U11'), array(['10', '15', '20'], dtype='<U11')]


In [28]:
# 분류 모델을 이용한 결측값 대체
from sklearn.neighbors import KNeighborsClassifier
# 훈련 데이터 생성
X=np.array([[0,2.10,1.45],[1,1.18,1.33],[0,1.22,1.27],[1,-0.21,-1.19]])
X_with_nan=np.array([[np.nan,0.87,0.31],[np.nan,-0.67,-0.22]])
# KNN 학습기 생성
clf=KNeighborsClassifier(3,weights='distance')
# 첫 번째 데이터가 레이블이고 나머지 데이터를 feature로 설정해서 훈련하기
# data, target 순으로 적어두었구나?
trained_model=clf.fit(X[:,1:],X[:,0])
# 예측
imputed_values=trained_model.predict(X_with_nan[:,1:])
print(imputed_values)
# 예측한 데이터와 원본 데이터를 합치기
# 옆으로 합치면 hstack
X_with_imputed=np.hstack((imputed_values.reshape(-1,1),X_with_nan[:,1:]))
print(X_with_imputed)
# 결측치를 대체한 데이터와 훈련에 사용한 데이터를 합쳐보자.
# 위 아애로 합치면 vstack을 사용합니다.
result=np.vstack((X_with_imputed,X))
print(result)

[0. 1.]
[[ 0.    0.87  0.31]
 [ 1.   -0.67 -0.22]]
[[ 0.    0.87  0.31]
 [ 1.   -0.67 -0.22]
 [ 0.    2.1   1.45]
 [ 1.    1.18  1.33]
 [ 0.    1.22  1.27]
 [ 1.   -0.21 -1.19]]


In [31]:
# 가장 많이 나오는 데이터로 대체
from sklearn.impute import SimpleImputer
X_complete=np.vstack((X_with_nan,X))
print(X_complete)
imputer=SimpleImputer(strategy='most_frequent')
imputer.fit_transform(X_complete)

[[  nan  0.87  0.31]
 [  nan -0.67 -0.22]
 [ 0.    2.1   1.45]
 [ 1.    1.18  1.33]
 [ 0.    1.22  1.27]
 [ 1.   -0.21 -1.19]]


array([[ 0.  ,  0.87,  0.31],
       [ 0.  , -0.67, -0.22],
       [ 0.  ,  2.1 ,  1.45],
       [ 1.  ,  1.18,  1.33],
       [ 0.  ,  1.22,  1.27],
       [ 1.  , -0.21, -1.19]])

### 정규표현식

In [33]:
import re
# 매칭 여부를 확인하자
match=re.match('[0-9]', '1234')
# 패턴에 일치하는 데이터가 있으면 match 객체 리턴
# 없으면 None 리턴
print(match)
match2=re.match('[0-9]', 'abc')
print(match2)

<re.Match object; span=(0, 1), match='1'>
None


In [37]:
string='@안녕하세요 반갑습니다^^ 486 의미없는 숫자는...!!!'
# 숫자 데이터 제거하기
p=re.compile('[0-9]+')
result=p.sub('',string)
print(result)
# 특수문자 제거하기
p=re.compile('\W+')
result=p.sub(' ',result)
print(result)

@안녕하세요 반갑습니다^^  의미없는 숫자는...!!!
 안녕하세요 반갑습니다 의미없는 숫자는 


In [39]:
import unicodedata
import sys
text_data=['안녕하세요 반갑습니다.', 'My job is Programmer.', 'C&C++ ,python']
# 구두점 딕셔너리를 생성
punctuation=dict.fromkeys(i for i in range(sys.maxunicode)
                         if unicodedata.category(chr(i)).startswith('P'))
result=[string.translate(punctuation) for string in text_data]
print(result)

['안녕하세요 반갑습니다', 'My job is Programmer', 'CC++ python']


In [40]:
!pip install nltk

In [41]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [44]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
string='The science of today is the technology of tomorrow'
print(word_tokenize(string)) 
string="품질은 양보다 중요합니다 한 번의 홈런이 두번의 2루타보다 낫습니다. 혁신은 현존하는 수천 가지 것들 중에 아니라고 말하는 것이다."
print(sent_tokenize(string))

['The', 'science', 'of', 'today', 'is', 'the', 'technology', 'of', 'tomorrow']
['품질은 양보다 중요합니다 한 번의 홈런이 두번의 2루타보다 낫습니다.', '혁신은 현존하는 수천 가지 것들 중에 아니라고 말하는 것이다.']


In [45]:
# 불용어 제거 - 한글은 불용어 사전이 없기에 직접 만들어야 한다.
word_korean=['1월','2월','3월','4월']
stopwords=['2월','3월']
print([i for i in word_korean if i not in stopwords])
# i for i in word_korean if i not in stopwords 는 작업을 수행해서
# generator를 생성. 이는 이터레이터로 접근할 수 있는 객체임

['1월', '4월']


In [49]:
# 영문도 해보자.
from nltk.corpus import stopwords
word_english=['chief','the','an','and','president','kenedy', 'move']
result=[w for w in word_english if not w in stopwords.words('english')]
print(result)
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
result=[w for w in word_english if not w in ENGLISH_STOP_WORDS]
print(result)

['chief', 'president', 'kenedy', 'move']
['chief', 'president', 'kenedy']


In [54]:
# 영문 어간 추출
from nltk.stem import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import word_tokenize
string="All pythoners have pythoned poorly at least once"
# 단어 토큰화
words=word_tokenize(string)
print(words)
# 어간 추출
ps_stemmer=PorterStemmer()
for w in words:
    print(ps_stemmer.stem(w), end='\t')
print()
# 어간 추출
ls_stemmer=LancasterStemmer()
for w in words:
    print(ls_stemmer.stem(w), end='\t')

['All', 'pythoners', 'have', 'pythoned', 'poorly', 'at', 'least', 'once']
all	python	have	python	poorli	at	least	onc	
al	python	hav	python	poor	at	least	ont	

In [59]:
# 형태소 분석 - 품사 태깅
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag
from nltk import word_tokenize
tokens=word_tokenize('The little yellow dog barked at th persian cat')
tags_en=pos_tag(tokens)
# 단어와 품사의 튜플 리스트로 출력
# 명사 NN, 고유명사 NNP, 부사 RB, 동사 VBD, 동명사 현재분사 VBG, 형용사 JJ
print(tags_en)
print()
#명사랑 고유명사만 뽑을래
print([word for word, tag in tags_en if tag in ['NN', 'NNP']])

[('The', 'DT'), ('little', 'JJ'), ('yellow', 'JJ'), ('dog', 'NN'), ('barked', 'VBD'), ('at', 'IN'), ('th', 'JJ'), ('persian', 'JJ'), ('cat', 'NN')]

['dog', 'cat']


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [61]:
!pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)


In [66]:
# 한글 형태소 분석
from konlpy.tag import Kkma
text='태양계는 지금으로부터 약 46억년 전 거대한 분자 구름의 일부분이 중력 붕괴를 일으키면서 형성되었다.'
# kkma=Kkma()
# #문장 분석
# print(kkma.sentences(text))
# # 단어 분석
# print(kkma.nouns(text))
# # 형태소 분석
# print(kkma.pos(text))
# 다른 형태소 분석기
from konlpy.tag import Hannanum
hannanum=Hannanum()
# 단어 분석
print(hannanum.nouns(text))
# 형태소 분석
print(hannanum.morphs(text))
print(hannanum.pos(text))

['태양계', '지금', '약', '46억년', '전', '거대', '자', '구름', '일부분', '중력', '붕괴', '형성']
['태양계', '는', '지금', '으로부터', '약', '46억년', '전', '거대', '하', 'ㄴ', '불', 'ㄴ', '자', '구름', '의', '일부분', '이', '중력', '붕괴', '를', '일으키', '면서', '형성', '되', '었다', '.']
[('태양계', 'N'), ('는', 'J'), ('지금', 'N'), ('으로부터', 'J'), ('약', 'N'), ('46억년', 'N'), ('전', 'N'), ('거대', 'N'), ('하', 'X'), ('ㄴ', 'E'), ('불', 'P'), ('ㄴ', 'E'), ('자', 'N'), ('구름', 'N'), ('의', 'J'), ('일부분', 'N'), ('이', 'J'), ('중력', 'N'), ('붕괴', 'N'), ('를', 'J'), ('일으키', 'P'), ('면서', 'E'), ('형성', 'N'), ('되', 'X'), ('었다', 'E'), ('.', 'S')]


In [69]:
# BoW(Bag of Word) - 단어의 개수
from sklearn.feature_extraction.text import CountVectorizer
text_Data=np.array([
    'I love my puppy',
    'Sweden is good',
    'I hate japan',
    'Germany beats both'
])
count=CountVectorizer()
bag_of_words=count.fit_transform(text_Data)
# 이렇게 부르면 희소행렬
# print(bag_of_words)
# 밀집 행렬의 형태로 출력하려면
print(count.get_feature_names_out())
print(bag_of_words.toarray())

['beats' 'both' 'germany' 'good' 'hate' 'is' 'japan' 'love' 'my' 'puppy'
 'sweden']
[[0 0 0 0 0 0 0 1 1 1 0]
 [0 0 0 1 0 1 0 0 0 0 1]
 [0 0 0 0 1 0 1 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0 0]]


In [73]:
# tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
text_Data=np.array([
    'I love my puppy',
    'Sweden is good',
    'l puppy love',
    'Germany good',
    'Germany beats both'
])
# tf-idf 객체 생성
tfidf=TfidfVectorizer()
feature_matrix=tfidf.fit_transform(text_Data)
# print(feature_matrix)
print(tfidf.vocabulary_)
print(feature_matrix.toarray())

{'love': 5, 'my': 6, 'puppy': 7, 'sweden': 8, 'is': 4, 'good': 3, 'germany': 2, 'beats': 0, 'both': 1}
[[0.         0.         0.         0.         0.         0.53177225
  0.659118   0.53177225 0.        ]
 [0.         0.         0.         0.49552379 0.61418897 0.
  0.         0.         0.61418897]
 [0.         0.         0.         0.         0.         0.70710678
  0.         0.70710678 0.        ]
 [0.         0.         0.70710678 0.70710678 0.         0.
  0.         0.         0.        ]
 [0.61418897 0.61418897 0.49552379 0.         0.         0.
  0.         0.         0.        ]]
